In [1]:
from models.mvsnet import CascadeMVSNet
from utils import load_ckpt
from inplace_abn import ABN
from losses import SL1Loss
model = CascadeMVSNet(n_depths=[8,32,48],
                      interval_ratios=[1.0,2.0,4.0],
                      num_groups=1,
                      norm_act=ABN).cuda()
load_ckpt(model, 'ckpts/_ckpt_epoch_10.ckpt')
model.eval()

CascadeMVSNet(
  (feature): FeatureNet(
    (conv0): Sequential(
      (0): ConvBnReLU(
        (conv): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): ABN(8, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
      )
      (1): ConvBnReLU(
        (conv): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): ABN(8, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
      )
    )
    (conv1): Sequential(
      (0): ConvBnReLU(
        (conv): Conv2d(8, 16, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
        (bn): ABN(16, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
      )
      (1): ConvBnReLU(
        (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): ABN(16, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
      )
      (2): ConvBnReLU(
        (conv): Conv2d(16, 16, k

In [2]:
def abs_error(depth_pred, depth_gt, mask):
    depth_pred, depth_gt = depth_pred[mask], depth_gt[mask]
    return (depth_pred - depth_gt).abs()

def acc_threshold(depth_pred, depth_gt, mask, threshold):
    """
    computes the percentage of pixels whose depth error is less than @threshold
    """
    errors = abs_error(depth_pred, depth_gt, mask)
    acc_mask = errors < threshold
    return acc_mask.float()

# use abs

In [4]:
from torch.utils.data import Dataset
from datasets.utils import read_pfm
import os
import numpy as np
import cv2
from PIL import Image
import torch
from torchvision import transforms as T

class DTUDataset(Dataset):
    def __init__(self, root_dir, split,model, n_views=3, levels=3, depth_interval=2.65,
                 img_wh=None,abs_error ="abs"):
        """
        img_wh should be set to a tuple ex: (1152, 864) to enable test mode!
        """
        self.root_dir = root_dir
        self.split = split
        assert self.split in ['train', 'val', 'test'], \
            'split must be either "train", "val" or "test"!'
        self.img_wh = img_wh
        if img_wh is not None:
            assert img_wh[0]%32==0 and img_wh[1]%32==0, \
                'img_wh must both be multiples of 32!'
        self.build_metas()
        self.n_views = n_views
        self.levels = levels # FPN levels
        self.depth_interval = depth_interval
        self.build_proj_mats()
        self.define_transforms()
        self.model = model.cuda()
        self.model.device = "cuda"
        self.loss = SL1Loss()
        
        
    def build_metas(self):
        self.metas = []
        with open(f'/root/autodl-tmp/project/dp_simple/CasMVSNet_pl/datasets/lists/dtu/{self.split}.txt') as f:
            self.scans = [line.rstrip() for line in f.readlines()]
        self.output_pkl = f'/root/autodl-tmp/project/dp_simple/CasMVSNet_pl/datasets/lists/dtu/{self.split}_abs.pkl'

        # light conditions 0-6 for training
        # light condition 3 for testing (the brightest?)
        light_idxs = list(range(7))

        pair_file = "Cameras/pair.txt"
        for scan in self.scans:
            with open(os.path.join(self.root_dir, pair_file)) as f:
                num_viewpoint = int(f.readline())
                # viewpoints (49)
                for _ in range(num_viewpoint):
                    ref_view = int(f.readline().rstrip())
                    src_views = [int(x) for x in f.readline().rstrip().split()[1::2]]
                    self.metas += [(scan, ref_view, src_views)]

                    # for light_idx in light_idxs:
                    #     
    def build_proj_mats(self):
        proj_mats = []
        for vid in range(49): # total 49 view ids
            if self.img_wh is None:
                proj_mat_filename = os.path.join(self.root_dir,
                                                 f'Cameras/train/{vid:08d}_cam.txt')
            else:
                proj_mat_filename = os.path.join(self.root_dir,
                                                 f'Cameras/{vid:08d}_cam.txt')
            intrinsics, extrinsics, depth_min = \
                self.read_cam_file(proj_mat_filename)
            if self.img_wh is not None: # resize the intrinsics to the coarsest level
                intrinsics[0] *= self.img_wh[0]/1600/4
                intrinsics[1] *= self.img_wh[1]/1200/4

            # multiply intrinsics and extrinsics to get projection matrix
            proj_mat_ls = []
            for l in reversed(range(self.levels)):
                proj_mat_l = np.eye(4)
                proj_mat_l[:3, :4] = intrinsics @ extrinsics[:3, :4]
                intrinsics[:2] *= 2 # 1/4->1/2->1
                proj_mat_ls += [torch.FloatTensor(proj_mat_l)]
            # (self.levels, 4, 4) from fine to coarse
            proj_mat_ls = torch.stack(proj_mat_ls[::-1])
           
            proj_mats += [(proj_mat_ls, depth_min)]

        self.proj_mats = proj_mats

    def read_cam_file(self, filename):
        with open(filename) as f:
            lines = [line.rstrip() for line in f.readlines()]
        # extrinsics: line [1,5), 4x4 matrix
        extrinsics = np.fromstring(' '.join(lines[1:5]), dtype=np.float32, sep=' ')
        extrinsics = extrinsics.reshape((4, 4))
        # intrinsics: line [7-10), 3x3 matrix
        intrinsics = np.fromstring(' '.join(lines[7:10]), dtype=np.float32, sep=' ')
        intrinsics = intrinsics.reshape((3, 3))
        # depth_min & depth_interval: line 11
        depth_min = float(lines[11].split()[0])
        return intrinsics, extrinsics, depth_min

    def read_depth(self, filename):
        depth = np.array(read_pfm(filename)[0], dtype=np.float32) # (1200, 1600)
        if self.img_wh is None:
            depth = cv2.resize(depth, None, fx=0.5, fy=0.5,
                            interpolation=cv2.INTER_NEAREST) # (600, 800)
            depth_0 = depth[44:556, 80:720] # (512, 640)
        else:
            depth_0 = cv2.resize(depth, self.img_wh,
                                 interpolation=cv2.INTER_NEAREST)
        depth_1 = cv2.resize(depth_0, None, fx=0.5, fy=0.5,
                             interpolation=cv2.INTER_NEAREST)
        depth_2 = cv2.resize(depth_1, None, fx=0.5, fy=0.5,
                             interpolation=cv2.INTER_NEAREST)

        depths = {"level_0": torch.FloatTensor(depth_0),
                  "level_1": torch.FloatTensor(depth_1),
                  "level_2": torch.FloatTensor(depth_2)}
        
        return depths

    def read_mask(self, filename):
        mask = cv2.imread(filename, 0) # (1200, 1600)
       
        if self.img_wh is None:
            mask = cv2.resize(mask, None, fx=0.5, fy=0.5,
                            interpolation=cv2.INTER_NEAREST) # (600, 800)
            mask_0 = mask[44:556, 80:720] # (512, 640)
        else:
            mask_0 = cv2.resize(mask, self.img_wh,
                                interpolation=cv2.INTER_NEAREST)
        mask_1 = cv2.resize(mask_0, None, fx=0.5, fy=0.5,
                            interpolation=cv2.INTER_NEAREST)
        mask_2 = cv2.resize(mask_1, None, fx=0.5, fy=0.5,
                            interpolation=cv2.INTER_NEAREST)

        masks = {"level_0": torch.BoolTensor(mask_0),
                 "level_1": torch.BoolTensor(mask_1),
                 "level_2": torch.BoolTensor(mask_2)}

        return masks

    def define_transforms(self):
        if self.split == 'train': # you can add augmentation here
            self.transform = T.Compose([T.ToTensor(),
                                        T.Normalize(mean=[0.485, 0.456, 0.406], 
                                                    std=[0.229, 0.224, 0.225]),
                                       ])
        else:
            self.transform = T.Compose([T.ToTensor(),
                                        T.Normalize(mean=[0.485, 0.456, 0.406], 
                                                    std=[0.229, 0.224, 0.225]),
                                       ])
        self.unpreprocess = T.Compose([
            T.Normalize(mean=[0, 0, 0], std=[1/0.229, 1/0.224, 1/0.225]),
            T.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
        ])
    
    def decode_batch(self, batch):
        imgs = batch['imgs']
        proj_mats = batch['proj_mats']
        depths = batch['depths']
        masks = batch['masks']
        init_depth_min = batch['init_depth_min']
        depth_interval = batch['depth_interval']
        return imgs, proj_mats, depths, masks, init_depth_min, depth_interval

    def __len__(self):
        return len(self.metas)


    def runs(self):
        import pickle as pkl
        dictionary = {}
        from tqdm import tqdm
        print("length is",len(self.metas))
        scans = {}
        for idx in tqdm(range(len(self.metas))):
            index = self.runonce(idx,dictionary,scans)
            print(index)

        with open(self.output_pkl,'wb') as f:
            pkl.dump(dictionary,f)
        return scans


    def runonce(self, idx,dictionary,scans):
       
        scan, ref_view, src_views = self.metas[idx]
        # use only the reference view and first nviews-1 source views
        view_ids = [ref_view] + src_views[:self.n_views-1]

        output_key = f"{scan}_{ref_view}_{src_views[0]}_{src_views[1]}"
        if scan not in scans.keys():
            scans[scan]=[]
        losses =[]
        for light_idx in range(7):
            sample = {}
            imgs = []
            cams = []
            proj_mats = []
            
            for i, vid in enumerate(view_ids):
            # NOTE that the id in image file names is from 1 to 49 (not 0~48)
            
                img_filename = os.path.join(self.root_dir,
                                f'Rectified/{scan}_train/rect_{vid+1:03d}_{light_idx}_r5000.png')
                mask_filename = os.path.join(self.root_dir,
                                f'Depths/{scan}/depth_visual_{vid:04d}.png')
                depth_filename = os.path.join(self.root_dir,
                                f'Depths/{scan}/depth_map_{vid:04d}.pfm')
        

                img = Image.open(img_filename)
                if self.img_wh is not None:
                    img = img.resize(self.img_wh, Image.BILINEAR)
                img = self.transform(img)
                imgs += [img]

                proj_mat_ls, depth_min = self.proj_mats[vid]
            



                if i == 0:  # reference view
                    
                    sample['init_depth_min'] = torch.FloatTensor([depth_min]).unsqueeze(0).to(self.model.device)
                    
                    sample['masks'] = self.read_mask(mask_filename)
                    for key in sample['masks']:
                        sample['masks'][key] = sample['masks'][key].unsqueeze(0).to(self.model.device)
                    sample['depths'] = self.read_depth(depth_filename)
                    for key in sample['depths']:
                        sample['depths'][key] = sample['depths'][key].unsqueeze(0).to(self.model.device)
                    sample["depth"] = sample["depths"]["level_0"]
                    ref_proj_inv = torch.inverse(proj_mat_ls)
                else:
                    
                    proj_mats += [proj_mat_ls @ ref_proj_inv]
       
        
            imgs = torch.stack(imgs) # (V, 3, H, W)
            proj_mats = torch.stack(proj_mats)[:,:,:3] # (V-1, self.levels, 3, 4) from fine to coarse
            
            
        
            sample['imgs'] = imgs.unsqueeze(0).to(self.model.device)
            sample['proj_mats'] = proj_mats.unsqueeze(0).to(self.model.device)
            sample['depth_interval'] = torch.FloatTensor([self.depth_interval]).unsqueeze(0).to(self.model.device)
            sample['scan_vid'] = (scan, ref_view)

            imgs, proj_mats, depths, masks, init_depth_min, depth_interval = self.decode_batch(sample)
            with torch.no_grad():
                
                results = self.model(imgs, proj_mats, init_depth_min, depth_interval)
                gt_depths = depths["level_0"]
                pred_depths = results["depth_0"]
                mask = masks["level_0"]
                loss = abs_error(gt_depths,pred_depths, mask).mean()
               
                losses.append(loss.item())
        # save the minimum loss from list losses
        print(losses)
        index = np.argmin(np.array(losses))
        dictionary[output_key] =  index
        scans[scan].append(index)




            

        return index

In [5]:
path_val = "/root/autodl-tmp/project/dp_simple/CasMVSNet_pl/datasets/lists/dtu/val_abs.pkl"
path_train =  "/root/autodl-tmp/project/dp_simple/CasMVSNet_pl/datasets/lists/dtu/train_abs.pkl"
path_test = "/root/autodl-tmp/project/dp_simple/CasMVSNet_pl/datasets/lists/dtu/test_abs.pkl"

In [6]:
import pickle
with open(path_val,"rb") as f:
    abs_val = pickle.load(f)
with open(path_test,"rb") as f:
    abs_test = pickle.load(f)
with open(path_train,"rb") as f:
    abs_train = pickle.load(f)

In [7]:
import matplotlib.pyplot as plt

In [17]:

scans_val = {}
for scans in abs_val:
    key = scans.split('_')[0]
    if key not in scans_val:
        scans_val[key] = []
    scans_val[key].append(abs_val[scans])
for scan in scans_val.keys():
    scans_val[scan] = np.stack(scans_val[scan])
    print(scans_val[scan].shape)
    print(np.mean(scans_val[scan],axis = 0)
          ,np.min(np.mean(scans_val[scan],axis = 0))/np.max(np.mean(scans_val[scan],axis = 0)))



(49, 7)
[3.05263341 2.92222905 2.79727794 2.86347311 2.84872966 2.86011306
 2.89029897] 0.9163491210564708
(49, 7)
[1.27925969 1.21662743 1.19024779 1.17714057 1.17585228 1.13693983
 1.13944231] 0.8887482659126668
(49, 7)
[4.23918102 4.3646479  4.26873698 4.152941   4.16164234 4.44407791
 4.15546879] 0.9344887917998775
(49, 7)
[6.7044504  6.78296111 6.79752936 6.78882487 6.74292021 6.64370545
 6.71224226] 0.9773706147046067
(49, 7)
[8.33500304 8.26974845 8.08564879 7.80853114 7.63656337 7.53135754
 7.33638762] 0.8801901560139066
(49, 7)
[1.18667069 1.32775589 1.20760566 1.06743625 1.15983699 0.91010288
 0.87788923] 0.6611827016405877
(49, 7)
[21.1885944  21.17826699 21.25191359 21.18731575 20.97766127 20.14655309
 19.91200681] 0.936951240901804
(49, 7)
[1.69360119 1.60652492 1.57134287 1.54112932 1.56236016 1.63116413
 1.62020702] 0.9099717997665584
(49, 7)
[3.64535844 3.69948315 3.71793726 3.69029421 3.68635351 3.71370282
 3.58273512] 0.9636351756172192
(49, 7)
[2.2589177  2.24700747 

In [1]:
dtu_data = DTUDataset(root_dir="/root/autodl-tmp/mvs_training/dtu",split="val",model=model)
scan_index = dtu_data.runs()


NameError: name 'DTUDataset' is not defined

In [ ]:
scan_index.keys()
import matplotlib.pyplot as plt
for scan in scan_index.keys():
    print(scan)
    indexes = scan_index[scan]
    plt.hist(indexes)
    plt.show()

NameError: name 'scan_index' is not defined

In [19]:
dtu_data = DTUDataset(root_dir="/root/autodl-tmp/mvs_training/dtu",split="test",model=model)
dtu_data.runs()

length is 1078


  0%|          | 1/1078 [00:01<23:04,  1.29s/it]

[1.356486439704895, 1.1690418720245361, 1.158003330230713, 1.1440080404281616, 1.1422415971755981, 1.3979743719100952, 1.4113914966583252]
4


  0%|          | 2/1078 [00:02<22:41,  1.27s/it]

[0.7865393161773682, 0.8312621116638184, 0.7722406387329102, 0.7643179297447205, 0.7747400999069214, 0.7159979343414307, 0.690311074256897]
6


...

[1.2156598567962646, 1.052027940750122, 0.9620014429092407, 0.9186756610870361, 0.9410143494606018, 0.9926929473876953, 1.0012750625610352]
3


100%|█████████▉| 1077/1078 [22:26<00:01,  1.40s/it]

[1.1659046411514282, 1.0459810495376587, 1.0091638565063477, 0.9704414010047913, 1.0940701961517334, 1.2271963357925415, 1.236135721206665]
3


100%|██████████| 1078/1078 [22:28<00:00,  1.25s/it]

[1.2431268692016602, 1.1234214305877686, 1.069701910018921, 1.0949561595916748, 1.1587471961975098, 1.798396110534668, 1.6840637922286987]
2


In [20]:
dtu_data = DTUDataset(root_dir="/root/autodl-tmp/mvs_training/dtu",split="train",model=model)
dtu_data.runs()

length is 3871


  0%|          | 1/3871 [00:02<2:16:08,  2.11s/it]

[2.5071258544921875, 2.4161105155944824, 2.3703043460845947, 2.3676583766937256, 2.383385181427002, 2.490417957305908, 2.4183969497680664]
3


  0%|          | 2/3871 [00:03<1:55:14,  1.79s/it]

[0.843677282333374, 0.9084333181381226, 0.8315354585647583, 0.8473768830299377, 0.8860723376274109, 0.9538230895996094, 0.8743266463279724]
2


...

[2.219911813735962, 1.6170945167541504, 1.7016537189483643, 1.7638508081436157, 1.861127257347107, 1.779179573059082, 1.7936310768127441]
1


100%|█████████▉| 3870/3871 [1:21:19<00:01,  1.26s/it]

[2.792257070541382, 2.830446481704712, 2.712644100189209, 2.701533317565918, 2.840411901473999, 2.9877753257751465, 2.8973801136016846]
3


100%|██████████| 3871/3871 [1:21:20<00:00,  1.26s/it]

[1.5153565406799316, 1.4463021755218506, 1.5580074787139893, 1.5432465076446533, 1.6882375478744507, 1.8323699235916138, 1.9269875288009644]
1


In [24]:
import pickle as pkl
with open(dtu_data.output_pkl,'rb') as f:
    dictionary = pkl.load(f)

In [25]:
dictionary

{'scan2_0_10_1': 3,
 'scan2_1_9_10': 2,
 'scan2_2_8_1': 6,
 'scan2_3_7_6': 6,
 'scan2_4_5_6': 4,
 'scan2_5_6_18': 6,
 'scan2_6_17_7': 6,
 'scan2_7_16_8': 1,
 'scan2_8_15_9': 1,
 'scan2_9_13_14': 2,
 'scan2_10_12_9': 0,
 'scan2_11_12_27': 6,
 'scan2_12_27_10': 1,
 'scan2_13_12_26': 2,
 'scan2_14_13_24': 5,
 'scan2_15_23_14': 6,
 'scan2_16_15_7': 5,
 'scan2_17_21_16': 0,
 'scan2_18_17_20': 6,
 'scan2_19_20_18': 2,
 'scan2_20_37_21': 3,
 'scan2_21_36_35': 3,
 'scan2_22_23_34': 6,
 'scan2_23_24_33': 2,
 'scan2_24_25_32': 6,
 'scan2_25_26_31': 1,
 'scan2_26_30_27': 2,
 'scan2_27_29_26': 3,
 'scan2_28_29_48': 1,
 'scan2_29_27_30': 0,
 'scan2_30_26_29': 3,
 'scan2_31_25_30': 2,
 'scan2_32_31_24': 6,
 'scan2_33_32_23': 1,
 'scan2_34_33_42': 6,
 'scan2_35_34_41': 5,
 'scan2_36_40_35': 3,
 'scan2_37_39_36': 5,
 'scan2_38_39_37': 5,
 'scan2_39_40_37': 3,
 'scan2_40_36_41': 4,
 'scan2_41_35_42': 5,
 'scan2_42_34_43': 4,
 'scan2_43_33_44': 1,
 'scan2_44_32_45': 6,
 'scan2_45_31_46': 5,
 'scan2_46_3

In [21]:
img_filename = os.path.join("/root/autodl-tmp/mvs_training/dtu/",
                                f'Rectified/scan77_train/rect_002_6_r5000.png')

In [22]:
img = Image.open(img_filename)

In [23]:
import numpy as np
np.array(img).shape

(512, 640, 3)